In [3]:
import numpy as np
from viewser import Queryset, Column
import pandas as pd
from pathlib import Path

qs_conflictology = (Queryset("conflictology", "country_month")

                    # target variable
                    .with_column(Column("ln_ged_sb", from_table="ged2_cm", from_column="ged_sb_best_sum_nokgi")
                                 .transform.ops.ln()
                                 .transform.missing.fill()
                                 )
                    )

# qs_conflictology.publish()


# In[270]:


df_conflictology = qs_conflictology.fetch()


# In[271]:


df_conflictology


# In[272]:


df_conflictology.reset_index(inplace=True)


# In[273]:


# create a dataframe with a particular country_id
df_conflictology_47 = df_conflictology[df_conflictology['country_id'] == 47]


# In[274]:


df_conflictology_47


# In[275]:


df_conflictology_47[f'step_pred_1'] = None


# In[276]:


df_conflictology_47


# In[277]:


len(df_conflictology_47)


# In[278]:


# Initialize the column with empty lists
df_conflictology_47['step_pred_1'] = [None]*len(df_conflictology_47)

# For each row in the DataFrame
for i in range(0, len(df_conflictology_47)):
    # Create a list of the ln_ged_sb values from the previous 12 months
    df_conflictology_47['step_pred_1'].iloc[i] = df_conflictology_47['ln_ged_sb'].iloc[i-12:i].values.tolist()


# In[279]:


df_conflictology_47.loc[df_conflictology_47['month_id']
                        == 483, 'step_pred_1'].values[0]


# In[280]:


df_conflictology_47.loc[df_conflictology_47['month_id']
                        == 482, 'step_pred_1'].values[0]


# In[281]:


df = df_conflictology_47


# In[282]:


df


# In[283]:


df = df.drop(columns="ln_ged_sb")


# In[284]:


df


# In[285]:


df = df.explode("step_pred_1").fillna(0).astype("int32")

df['draw'] = df.groupby(['month_id', 'country_id']).cumcount()
df.set_index(['month_id', 'country_id', 'draw'], inplace=True)


# In[286]:


df


# In[287]:


for i in range(2, 37):
    df[f'step_pred_{i}'] = df['step_pred_1'].shift(i-1)


# In[288]:


df


# In[289]:


df_conflictology_country = {}
for i in range(1, 247):
    df_conflictology_country[i] = df_conflictology[df_conflictology['country_id'] == i]


# In[290]:


df_conflictology_country


# In[291]:


for i in range(1, 247):
    df = df_conflictology_country[i]
    ln_ged_sb_values = df['ln_ged_sb'].tolist()
    step_pred_1_values = []

    for k in range(len(df)):
        if k < 12:
            step_pred_1_values.append(None)
        else:
            step_pred_1_values.append(ln_ged_sb_values[k-12:k])

    df['step_pred_1'] = step_pred_1_values


# In[292]:


df = df_conflictology_country[1]
selected_rows = df['step_pred_1'][df['month_id'] == 483]
count = sum(len(x) for x in selected_rows if x is not None)


# In[293]:


count


# In[294]:


for i in range(1, 247):
    df_conflictology_country[i] = df_conflictology_country[i].drop(
        columns="ln_ged_sb")
    df_conflictology_country[i] = df_conflictology_country[i].explode(
        "step_pred_1").fillna(0).astype("int32")
    df_conflictology_country[i]['draw'] = df_conflictology_country[i].groupby(
        ['month_id', 'country_id']).cumcount()
    df_conflictology_country[i].set_index(
        ['month_id', 'country_id', 'draw'], inplace=True)
    for x in range(2, 37):
        df_conflictology_country[i][f'step_pred_{x}'] = df_conflictology_country[i]['step_pred_1'].shift(x-1).fillna(0).astype("int32")


# In[295]:


df_conflictology_country[69]


# In[296]:


df_sss = df_conflictology_country[47]
# find values in step_pred_1 for a particular month_id
df_sss.reset_index(inplace=True)
month_id = 482
values = df_sss['step_pred_1'][df_sss['month_id'] == month_id]


# In[297]:


values


# In[298]:


df_sss = df_conflictology_country[47]
# find values in step_pred_1 for a particular month_id
df_sss.reset_index(inplace=True)
month_id = 482
values = df_sss['step_pred_1'][df_sss['month_id'] == month_id]


# In[299]:


values


# In[300]:


df_conflictology_country


# In[301]:


for i in range(1, 247):
    df_conflictology_country[i].reset_index(inplace=True)


# In[302]:


df_conflictology_country


# In[303]:


df_all = pd.concat(df_conflictology_country.values())


# In[304]:


df_all


# In[305]:


df_all.set_index(['month_id', 'country_id', 'draw'], inplace=True)


# In[306]:


df_all.drop(columns="level_0", inplace=True)
df_all.drop(columns="index", inplace=True)


# In[307]:


df_all


# In[308]:


# Assuming df_conflictology_country is a DataFrame
df_forecast = df_all
df_forecast.reset_index(inplace=True)
forecast = df_forecast[(df_forecast['month_id'] >= 481) & (
    df_forecast['month_id'] <= 481+35)]


# In[310]:


forecast.set_index(['month_id', 'country_id', 'draw'], inplace=True)


# In[ ]:


# forecast.drop(columns="level_0", inplace=True)


# In[312]:


forecast


# ### The above is an algorithm _______________

 .    

/var/folders/qx/vpttxp096k9bhf2r8qcpdxlm0000gn/T/ipykernel_5089/785918449.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_conflictology_47[f'step_pred_1'] = None
/var/folders/qx/vpttxp096k9bhf2r8qcpdxlm0000gn/T/ipykernel_5089/785918449.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_conflictology_47['step_pred_1'] = [None]*len(df_conflictology_47)
/var/folders/qx/vpttxp096k9bhf2r8qcpdxlm0000gn/T/ipykernel_5089/785918449.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of

step_pred_1  step_pred_2  step_pred_3  step_pred_4  \
month_id country_id draw                                                       
481      1          0               0            0            0            0   
                    1               0            0            0            0   
                    2               0            0            0            0   
                    3               0            0            0            0   
                    4               0            0            0            0   
...                               ...          ...          ...          ...   
516      246        7               2            1            0            0   
                    8               1            2            1            0   
                    9               0            1            2            1   
                    10              0            0            1            2   
                    11              0            0            0            1   

                          step_pred_5  step_pred_6  step_pred_7  step_pred_8  \
month_id country_id draw                                                       
481      1          0               0            0            0            0   
                    1               0            0            0            0   
                    2               0            0            0            0   
                    3               0            0            0            0   
                    4               0            0            0            0   
...                               ...          ...          ...          ...   
516      246        7               0            0            0            4   
                    8               0            0            0            0   
                    9               0            0            0            0   
                    10              1            0            0            0   
                    11              2            1            0            0   

                          step_pred_9  step_pred_10  ...  step_pred_27  \
month_id country_id draw                             ...                 
481      1          0               0             0  ...             0   
                    1               0             0  ...             0   
                    2               0             0  ...             0   
                    3               0             0  ...             0   
                    4               0             0  ...             0   
...                               ...           ...  ...           ...   
516      246        7               0             0  ...             0   
                    8               4             0  ...             0   
                    9               0             4  ...             0   
                    10              0             0  ...             1   
                    11              0             0  ...             2   

                          step_pred_28  step_pred_29  step_pred_30  \
month_id country_id draw                                             
481      1          0                0             0             0   
                    1                0             0             0   
                    2                0             0             0   
                    3                0             0             0   
                    4                0             0             0   
...                                ...           ...           ...   
516      246        7                0             0             4   
                    8                0             0             0   
                    9                0             0             0   
                    10               0             0             0   
                    11               1             0             0   

                          step_pred_31  step_pred_32  step_pred_33  \


In [ ]:
'future_partitioner_dict': {"train": (121, 492), "predict": (493, 504)}


In [10]:
from viewser import Queryset, Column
from pathlib import Path
import pandas as pd
import numpy as np


def views_conflictology_forecast(start_month_of_forecast):
    qs_conflictology = (Queryset("conflictology", "country_month")

                        # target variable
                        .with_column(Column("ln_ged_sb", from_table="ged2_cm", from_column="ged_sb_best_sum_nokgi")
                                    .transform.ops.ln()
                                    .transform.missing.fill()
                                    )
                        )

    # qs_conflictology.publish()


    # In[270]:


    df_conflictology = qs_conflictology.fetch()


    # In[271]:


    df_conflictology


    # In[272]:


    df_conflictology.reset_index(inplace=True)


    # In[273]:


    # create a dataframe with a particular country_id
    df_conflictology_47 = df_conflictology[df_conflictology['country_id'] == 47]


    # In[274]:


    df_conflictology_47


    # In[275]:


    df_conflictology_47[f'step_pred_1'] = None


    # In[276]:


    df_conflictology_47


    # In[277]:


    len(df_conflictology_47)


    # In[278]:


    # Initialize the column with empty lists
    df_conflictology_47['step_pred_1'] = [None]*len(df_conflictology_47)

    # For each row in the DataFrame
    for i in range(0, len(df_conflictology_47)):
        # Create a list of the ln_ged_sb values from the previous 12 months
        df_conflictology_47['step_pred_1'].iloc[i] = df_conflictology_47['ln_ged_sb'].iloc[i-12:i].values.tolist()


    # In[279]:


    df_conflictology_47.loc[df_conflictology_47['month_id']
                            == 483, 'step_pred_1'].values[0]


    # In[280]:


    df_conflictology_47.loc[df_conflictology_47['month_id']
                            == 482, 'step_pred_1'].values[0]


    # In[281]:


    df = df_conflictology_47


    # In[282]:


    df


    # In[283]:


    df = df.drop(columns="ln_ged_sb")


    # In[284]:


    df


    # In[285]:


    df = df.explode("step_pred_1").fillna(0).astype("int32")

    df['draw'] = df.groupby(['month_id', 'country_id']).cumcount()
    df.set_index(['month_id', 'country_id', 'draw'], inplace=True)


    # In[286]:


    df


    # In[287]:


    for i in range(2, 37):
        df[f'step_pred_{i}'] = df['step_pred_1'].shift(i-1)


    # In[288]:


    df


    # In[289]:


    df_conflictology_country = {}
    for i in range(1, 247):
        df_conflictology_country[i] = df_conflictology[df_conflictology['country_id'] == i]


    # In[290]:


    df_conflictology_country


    # In[291]:


    for i in range(1, 247):
        df = df_conflictology_country[i]
        ln_ged_sb_values = df['ln_ged_sb'].tolist()
        step_pred_1_values = []

        for k in range(len(df)):
            if k < 12:
                step_pred_1_values.append(None)
            else:
                step_pred_1_values.append(ln_ged_sb_values[k-12:k])

        df['step_pred_1'] = step_pred_1_values


    # In[292]:


    df = df_conflictology_country[1]
    selected_rows = df['step_pred_1'][df['month_id'] == 483]
    count = sum(len(x) for x in selected_rows if x is not None)


    # In[293]:


    count


    # In[294]:


    for i in range(1, 247):
        df_conflictology_country[i] = df_conflictology_country[i].drop(
            columns="ln_ged_sb")
        df_conflictology_country[i] = df_conflictology_country[i].explode(
            "step_pred_1").fillna(0).astype("int32")
        df_conflictology_country[i]['draw'] = df_conflictology_country[i].groupby(
            ['month_id', 'country_id']).cumcount()
        df_conflictology_country[i].set_index(
            ['month_id', 'country_id', 'draw'], inplace=True)
        for x in range(2, 37):
            df_conflictology_country[i][f'step_pred_{x}'] = df_conflictology_country[i]['step_pred_1'].shift(x-1).fillna(0).astype("int32")


    # In[295]:


    df_conflictology_country[69]


    # In[296]:


    df_sss = df_conflictology_country[47]
    # find values in step_pred_1 for a particular month_id
    df_sss.reset_index(inplace=True)
    month_id = 482
    values = df_sss['step_pred_1'][df_sss['month_id'] == month_id]


    # In[297]:


    values


    # In[298]:


    df_sss = df_conflictology_country[47]
    # find values in step_pred_1 for a particular month_id
    df_sss.reset_index(inplace=True)
    month_id = 482
    values = df_sss['step_pred_1'][df_sss['month_id'] == month_id]


    # In[299]:


    values


    # In[300]:


    df_conflictology_country


    # In[301]:


    for i in range(1, 247):
        df_conflictology_country[i].reset_index(inplace=True)


    # In[302]:


    df_conflictology_country


    # In[303]:


    df_all = pd.concat(df_conflictology_country.values())


    # In[304]:


    df_all


    # In[305]:


    df_all.set_index(['month_id', 'country_id', 'draw'], inplace=True)


    # In[306]:


    df_all.drop(columns="level_0", inplace=True)
    df_all.drop(columns="index", inplace=True)


    # In[307]:


    df_all


    # In[308]:


    # Assuming df_conflictology_country is a DataFrame
    df_forecast = df_all
    df_forecast.reset_index(inplace=True)
    forecast = df_forecast[(df_forecast['month_id'] >= start_month_of_forecast) & (
        df_forecast['month_id'] <= start_month_of_forecast+35)]


    # In[310]:


    forecast.set_index(['month_id', 'country_id', 'draw'], inplace=True)


    # In[ ]:


    # forecast.drop(columns="level_0", inplace=True)


    # In[312]:


    forecast


    # ### The above is an algorithm _______________
    return forecast

In [15]:
df = views_conflictology_forecast(850)

 .    

/var/folders/qx/vpttxp096k9bhf2r8qcpdxlm0000gn/T/ipykernel_5089/1767672064.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_conflictology_47[f'step_pred_1'] = None
/var/folders/qx/vpttxp096k9bhf2r8qcpdxlm0000gn/T/ipykernel_5089/1767672064.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_conflictology_47['step_pred_1'] = [None]*len(df_conflictology_47)
/var/folders/qx/vpttxp096k9bhf2r8qcpdxlm0000gn/T/ipykernel_5089/1767672064.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [16]:
df

step_pred_1  step_pred_2  step_pred_3  step_pred_4  \
month_id country_id draw                                                       
850      1          0               0            0            0            0   
                    1               0            0            0            0   
                    2               0            0            0            0   
                    3               0            0            0            0   
                    4               0            0            0            0   
...                               ...          ...          ...          ...   
852      246        7               0            0            0            0   
                    8               0            0            0            0   
                    9               0            0            0            0   
                    10              0            0            0            0   
                    11              0            0            0            0   

                          step_pred_5  step_pred_6  step_pred_7  step_pred_8  \
month_id country_id draw                                                       
850      1          0               0            0            0            0   
                    1               0            0            0            0   
                    2               0            0            0            0   
                    3               0            0            0            0   
                    4               0            0            0            0   
...                               ...          ...          ...          ...   
852      246        7               0            0            0            0   
                    8               0            0            0            0   
                    9               0            0            0            0   
                    10              0            0            0            0   
                    11              0            0            0            0   

                          step_pred_9  step_pred_10  ...  step_pred_27  \
month_id country_id draw                             ...                 
850      1          0               0             0  ...             0   
                    1               0             0  ...             0   
                    2               0             0  ...             0   
                    3               0             0  ...             0   
                    4               0             0  ...             0   
...                               ...           ...  ...           ...   
852      246        7               0             0  ...             0   
                    8               0             0  ...             0   
                    9               0             0  ...             0   
                    10              0             0  ...             0   
                    11              0             0  ...             0   

                          step_pred_28  step_pred_29  step_pred_30  \
month_id country_id draw                                             
850      1          0                0             0             0   
                    1                0             0             0   
                    2                0             0             0   
                    3                0             0             0   
                    4                0             0             0   
...                                ...           ...           ...   
852      246        7                0             0             0   
                    8                0             0             0   
                    9                0             0             0   
                    10               0             0             0   
                    11               0             0             0   

                          step_pred_31  step_pred_32  step_pred_33  \


In [ ]:
'future_partitioner_dict': {"train": (121, 492), "predict": (493, 504)}

#### A code for forecast as a partitioner dictionary

In [3]:
from viewser import Queryset, Column
from pathlib import Path
import pandas as pd
import numpy as np


def views_conflictology_forecast(start_month_of_forecast):
    qs_conflictology = (Queryset("conflictology", "country_month")

                        # target variable
                        .with_column(Column("ln_ged_sb", from_table="ged2_cm", from_column="ged_sb_best_sum_nokgi")
                                     .transform.ops.ln()
                                     .transform.missing.fill()
                                     )
                        )

    # qs_conflictology.publish()

    # In[270]:

    df_conflictology = qs_conflictology.fetch()

    # In[271]:

    df_conflictology

    # In[272]:

    df_conflictology.reset_index(inplace=True)

    # In[273]:

    # create a dataframe with a particular country_id
    df_conflictology_47 = df_conflictology[df_conflictology['country_id'] == 47]

    # In[274]:

    df_conflictology_47

    # In[275]:

    df_conflictology_47[f'step_pred_1'] = None

    # In[276]:

    df_conflictology_47

    # In[277]:

    len(df_conflictology_47)

    # In[278]:

    # Initialize the column with empty lists
    df_conflictology_47['step_pred_1'] = [None]*len(df_conflictology_47)

    # For each row in the DataFrame
    for i in range(0, len(df_conflictology_47)):
        # Create a list of the ln_ged_sb values from the previous 12 months
        df_conflictology_47['step_pred_1'].iloc[i] = df_conflictology_47['ln_ged_sb'].iloc[i-12:i].values.tolist()

    # In[279]:

    df_conflictology_47.loc[df_conflictology_47['month_id']
                            == 483, 'step_pred_1'].values[0]

    # In[280]:

    df_conflictology_47.loc[df_conflictology_47['month_id']
                            == 482, 'step_pred_1'].values[0]

    # In[281]:

    df = df_conflictology_47

    # In[282]:

    df

    # In[283]:

    df = df.drop(columns="ln_ged_sb")

    # In[284]:

    df

    # In[285]:

    df = df.explode("step_pred_1").fillna(0).astype("int32")

    df['draw'] = df.groupby(['month_id', 'country_id']).cumcount()
    df.set_index(['month_id', 'country_id', 'draw'], inplace=True)

    # In[286]:

    df

    # In[287]:

    for i in range(2, 37):
        df[f'step_pred_{i}'] = df['step_pred_1'].shift(i-1)

    # In[288]:

    df

    # In[289]:

    df_conflictology_country = {}
    for i in range(1, 247):
        df_conflictology_country[i] = df_conflictology[df_conflictology['country_id'] == i]

    # In[290]:

    df_conflictology_country

    # In[291]:

    for i in range(1, 247):
        df = df_conflictology_country[i]
        ln_ged_sb_values = df['ln_ged_sb'].tolist()
        step_pred_1_values = []

        for k in range(len(df)):
            if k < 12:
                step_pred_1_values.append(None)
            else:
                step_pred_1_values.append(ln_ged_sb_values[k-12:k])

        df['step_pred_1'] = step_pred_1_values

    # In[292]:

    df = df_conflictology_country[1]
    selected_rows = df['step_pred_1'][df['month_id'] == 483]
    count = sum(len(x) for x in selected_rows if x is not None)

    # In[293]:

    count

    # In[294]:

    for i in range(1, 247):
        df_conflictology_country[i] = df_conflictology_country[i].drop(
            columns="ln_ged_sb")
        df_conflictology_country[i] = df_conflictology_country[i].explode(
            "step_pred_1").fillna(0).astype("int32")
        df_conflictology_country[i]['draw'] = df_conflictology_country[i].groupby(
            ['month_id', 'country_id']).cumcount()
        df_conflictology_country[i].set_index(
            ['month_id', 'country_id', 'draw'], inplace=True)
        for x in range(2, 37):
            df_conflictology_country[i][f'step_pred_{x}'] = df_conflictology_country[i]['step_pred_1'].shift(x-1).fillna(0).astype("int32")

    # In[295]:

    df_conflictology_country[69]

    # In[296]:

    df_sss = df_conflictology_country[47]
    # find values in step_pred_1 for a particular month_id
    df_sss.reset_index(inplace=True)
    month_id = 482
    values = df_sss['step_pred_1'][df_sss['month_id'] == month_id]

    # In[297]:

    values

    # In[298]:

    df_sss = df_conflictology_country[47]
    # find values in step_pred_1 for a particular month_id
    df_sss.reset_index(inplace=True)
    month_id = 482
    values = df_sss['step_pred_1'][df_sss['month_id'] == month_id]

    # In[299]:

    values

    # In[300]:

    df_conflictology_country

    # In[301]:

    for i in range(1, 247):
        df_conflictology_country[i].reset_index(inplace=True)

    # In[302]:

    df_conflictology_country

    # In[303]:

    df_all = pd.concat(df_conflictology_country.values())

    # In[304]:

    df_all

    # In[305]:

    df_all.set_index(['month_id', 'country_id', 'draw'], inplace=True)

    # In[306]:

    df_all.drop(columns="level_0", inplace=True)
    df_all.drop(columns="index", inplace=True)

    # In[307]:

    df_all

    # In[308]:

    # Assuming df_conflictology_country is a DataFrame
    df_forecast = df_all
    df_forecast.reset_index(inplace=True)
    forecast = df_forecast[(df_forecast['month_id'] >= start_month_of_forecast[0]) & (
        df_forecast['month_id'] <= start_month_of_forecast[1])]

    # In[310]:

    forecast.set_index(['month_id', 'country_id', 'draw'], inplace=True)

    # In[ ]:

    # forecast.drop(columns="level_0", inplace=True)

    # In[312]:

    forecast

    # ### The above is an algorithm _______________
    return forecast

In [7]:
df = views_conflictology_forecast((422,504))

 .    

/var/folders/qx/vpttxp096k9bhf2r8qcpdxlm0000gn/T/ipykernel_7197/3357910387.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_conflictology_47[f'step_pred_1'] = None
/var/folders/qx/vpttxp096k9bhf2r8qcpdxlm0000gn/T/ipykernel_7197/3357910387.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_conflictology_47['step_pred_1'] = [None]*len(df_conflictology_47)
/var/folders/qx/vpttxp096k9bhf2r8qcpdxlm0000gn/T/ipykernel_7197/3357910387.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [8]:
df

step_pred_1  step_pred_2  step_pred_3  step_pred_4  \
month_id country_id draw                                                       
422      1          0               0            0            0            0   
                    1               0            0            0            0   
                    2               0            0            0            0   
                    3               0            0            0            0   
                    4               0            0            0            0   
...                               ...          ...          ...          ...   
504      246        7               1            2            2            0   
                    8               3            1            2            2   
                    9               4            3            1            2   
                    10              1            4            3            1   
                    11              0            1            4            3   

                          step_pred_5  step_pred_6  step_pred_7  step_pred_8  \
month_id country_id draw                                                       
422      1          0               0            0            0            0   
                    1               0            0            0            0   
                    2               0            0            0            0   
                    3               0            0            0            0   
                    4               0            0            0            0   
...                               ...          ...          ...          ...   
504      246        7               1            0            0            0   
                    8               0            1            0            0   
                    9               2            0            1            0   
                    10              2            2            0            1   
                    11              1            2            2            0   

                          step_pred_9  step_pred_10  ...  step_pred_27  \
month_id country_id draw                             ...                 
422      1          0               0             0  ...             0   
                    1               0             0  ...             0   
                    2               0             0  ...             0   
                    3               0             0  ...             0   
                    4               0             0  ...             0   
...                               ...           ...  ...           ...   
504      246        7               1             4  ...             1   
                    8               0             1  ...             0   
                    9               0             0  ...             2   
                    10              0             0  ...             2   
                    11              1             0  ...             1   

                          step_pred_28  step_pred_29  step_pred_30  \
month_id country_id draw                                             
422      1          0                0             0             0   
                    1                0             0             0   
                    2                0             0             0   
                    3                0             0             0   
                    4                0             0             0   
...                                ...           ...           ...   
504      246        7                0             0             0   
                    8                1             0             0   
                    9                0             1             0   
                    10               2             0             1   
                    11               2             2             0   

                          step_pred_31  step_pred_32  step_pred_33  \


#### A code to forecast using partitioner dictionary

In [16]:
from viewser import Queryset, Column
from pathlib import Path
import pandas as pd
import numpy as np


def views_conflictology_forecast(start_month_of_forecast,training_period):
    qs_conflictology = (Queryset("conflictology", "country_month")

                        # target variable
                        .with_column(Column("ln_ged_sb", from_table="ged2_cm", from_column="ged_sb_best_sum_nokgi")
                                     .transform.ops.ln()
                                     .transform.missing.fill()
                                     )
                        )

    # qs_conflictology.publish()

    # In[270]:

    df_conflictology = qs_conflictology.fetch()

    # In[271]:

    df_conflictology

    # In[272]:

    df_conflictology.reset_index(inplace=True)

    # In[273]:

    # create a dataframe with a particular country_id
    df_conflictology_47 = df_conflictology[df_conflictology['country_id'] == 47]

    # In[274]:

    df_conflictology_47

    # In[275]:

    df_conflictology_47[f'step_pred_1'] = None

    # In[276]:

    df_conflictology_47

    # In[277]:

    len(df_conflictology_47)

    # In[278]:

    # Initialize the column with empty lists
    df_conflictology_47['step_pred_1'] = [None]*len(df_conflictology_47)

    # For each row in the DataFrame
    for i in range(0, len(df_conflictology_47)):
        # Create a list of the ln_ged_sb values from the previous 12 months
        df_conflictology_47['step_pred_1'].iloc[i] = df_conflictology_47['ln_ged_sb'].iloc[i-12:i].values.tolist()

    # In[279]:

    df_conflictology_47.loc[df_conflictology_47['month_id']
                            == 483, 'step_pred_1'].values[0]

    # In[280]:

    df_conflictology_47.loc[df_conflictology_47['month_id']
                            == 482, 'step_pred_1'].values[0]

    # In[281]:

    df = df_conflictology_47

    # In[282]:

    df

    # In[283]:

    df = df.drop(columns="ln_ged_sb")

    # In[284]:

    df

    # In[285]:

    df = df.explode("step_pred_1").fillna(0).astype("int32")

    df['draw'] = df.groupby(['month_id', 'country_id']).cumcount()
    df.set_index(['month_id', 'country_id', 'draw'], inplace=True)

    # In[286]:

    df

    # In[287]:

    for i in range(2, 37):
        df[f'step_pred_{i}'] = df['step_pred_1'].shift(i-1)

    # In[288]:

    df

    # In[289]:

    df_conflictology_country = {}
    for i in range(1, 247):
        df_conflictology_country[i] = df_conflictology[df_conflictology['country_id'] == i]

    # In[290]:

    df_conflictology_country

    # In[291]:

    for i in range(1, 247):
        df = df_conflictology_country[i]
        ln_ged_sb_values = df['ln_ged_sb'].tolist()
        step_pred_1_values = []

        for k in range(len(df)):
            if k < 12:
                step_pred_1_values.append(None)
            else:
                step_pred_1_values.append(ln_ged_sb_values[k-(start_month_of_forecast[1]-training_period[1])-12:k-(start_month_of_forecast[1]-training_period[1])])

        df['step_pred_1'] = step_pred_1_values

    # In[292]:

    df = df_conflictology_country[1]
    selected_rows = df['step_pred_1'][df['month_id'] == 483]
    count = sum(len(x) for x in selected_rows if x is not None)

    # In[293]:

    count

    # In[294]:

    for i in range(1, 247):
        df_conflictology_country[i] = df_conflictology_country[i].drop(
            columns="ln_ged_sb")
        df_conflictology_country[i] = df_conflictology_country[i].explode(
            "step_pred_1").fillna(0).astype("int32")
        df_conflictology_country[i]['draw'] = df_conflictology_country[i].groupby(
            ['month_id', 'country_id']).cumcount()
        df_conflictology_country[i].set_index(
            ['month_id', 'country_id', 'draw'], inplace=True)
        for x in range(2, 37):
            df_conflictology_country[i][f'step_pred_{x}'] = df_conflictology_country[i]['step_pred_1'].shift(x-1).fillna(0).astype("int32")

    # In[295]:

    df_conflictology_country[69]

    # In[296]:

    df_sss = df_conflictology_country[47]
    # find values in step_pred_1 for a particular month_id
    df_sss.reset_index(inplace=True)
    month_id = 482
    values = df_sss['step_pred_1'][df_sss['month_id'] == month_id]

    # In[297]:

    values

    # In[298]:

    df_sss = df_conflictology_country[47]
    # find values in step_pred_1 for a particular month_id
    df_sss.reset_index(inplace=True)
    month_id = 482
    values = df_sss['step_pred_1'][df_sss['month_id'] == month_id]

    # In[299]:

    values

    # In[300]:

    df_conflictology_country

    # In[301]:

    for i in range(1, 247):
        df_conflictology_country[i].reset_index(inplace=True)

    # In[302]:

    df_conflictology_country

    # In[303]:

    df_all = pd.concat(df_conflictology_country.values())

    # In[304]:

    df_all

    # In[305]:

    df_all.set_index(['month_id', 'country_id', 'draw'], inplace=True)

    # In[306]:

    df_all.drop(columns="level_0", inplace=True)
    df_all.drop(columns="index", inplace=True)

    # In[307]:

    df_all

    # In[308]:

    # Assuming df_conflictology_country is a DataFrame
    df_forecast = df_all
    df_forecast.reset_index(inplace=True)
    forecast = df_forecast[(df_forecast['month_id'] >= start_month_of_forecast[0]) & (
        df_forecast['month_id'] <= start_month_of_forecast[1])]

    # In[310]:

    forecast.set_index(['month_id', 'country_id', 'draw'], inplace=True)

    # In[ ]:

    # forecast.drop(columns="level_0", inplace=True)

    # In[312]:

    forecast

    # ### The above is an algorithm _______________
    return forecast

In [17]:
df = views_conflictology_forecast((493,504),(121,492))

 .    

/var/folders/qx/vpttxp096k9bhf2r8qcpdxlm0000gn/T/ipykernel_7197/1728602409.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_conflictology_47[f'step_pred_1'] = None
/var/folders/qx/vpttxp096k9bhf2r8qcpdxlm0000gn/T/ipykernel_7197/1728602409.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_conflictology_47['step_pred_1'] = [None]*len(df_conflictology_47)
/var/folders/qx/vpttxp096k9bhf2r8qcpdxlm0000gn/T/ipykernel_7197/1728602409.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [18]:
df_forecast

step_pred_1  step_pred_2  step_pred_3  step_pred_4  \
month_id country_id draw                                                       
493      1          0               0            0            0            0   
                    1               0            0            0            0   
                    2               0            0            0            0   
                    3               0            0            0            0   
                    4               0            0            0            0   
...                               ...          ...          ...          ...   
504      246        7               2            1            2            0   
                    8               3            2            1            2   
                    9               0            3            2            1   
                    10              0            0            3            2   
                    11              2            0            0            3   

                          step_pred_5  step_pred_6  step_pred_7  step_pred_8  \
month_id country_id draw                                                       
493      1          0               0            0            0            0   
                    1               0            0            0            0   
                    2               0            0            0            0   
                    3               0            0            0            0   
                    4               0            0            0            0   
...                               ...          ...          ...          ...   
504      246        7               0            0            1            0   
                    8               0            0            0            1   
                    9               2            0            0            0   
                    10              1            2            0            0   
                    11              2            1            2            0   

                          step_pred_9  step_pred_10  ...  step_pred_27  \
month_id country_id draw                             ...                 
493      1          0               0             0  ...             0   
                    1               0             0  ...             0   
                    2               0             0  ...             0   
                    3               0             0  ...             0   
                    4               0             0  ...             0   
...                               ...           ...  ...           ...   
504      246        7               0             0  ...             0   
                    8               0             0  ...             0   
                    9               1             0  ...             2   
                    10              0             1  ...             1   
                    11              0             0  ...             2   

                          step_pred_28  step_pred_29  step_pred_30  \
month_id country_id draw                                             
493      1          0                0             0             0   
                    1                0             0             0   
                    2                0             0             0   
                    3                0             0             0   
                    4                0             0             0   
...                                ...           ...           ...   
504      246        7                0             1             0   
                    8                0             0             1   
                    9                0             0             0   
                    10               2             0             0   
                    11               1             2             0   

                          step_pred_31  step_pred_32  step_pred_33  \


In [19]:
df_forecast.to_parquet('df_forecast.parquet')